In [1]:
import os
import pandas as pd
from pandas.api.indexers import FixedForwardWindowIndexer
import warnings
warnings.filterwarnings('ignore')

In [41]:
folder_path = './data/' #未处理的文件所在文件夹
paras_path = './paras_data/' #处理后数据存储的位置
all_col =['trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount'] #文件中索要留下的列
ins_col = ['open','high','low','close'] #选择需要扩增的列
time_col ='trade_date' #用于排序的，股票每日交易时间
ins_time_col = [10,30,60,120,180,360,720] #扩增的时段

label_later_col = 'high'
label_font_col = 'open'
window_size = 7

In [44]:
# 遍历文件夹
for file_name in os.listdir(folder_path):
    # 判断文件扩展名是否为csv
    file_path = os.path.join(folder_path, file_name)
    if file_name.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    elif file_name.endswith('.csv'):
        df = pd.read_csv(file_path)
    df = df[all_col]
    df.dropna(inplace=True)
    df.sort_values(by='trade_date',inplace=True)
    df['trade_date'] =df['trade_date'].astype('string')
    df['trade_date'] =pd.to_datetime(df['trade_date'])
    df.reset_index(drop=True,inplace=True)
    future_drop = df.columns.to_list()

    tmp_col = ins_col
    time_col = ins_time_col
    ins_mean_col = list()
    ins_sum_col = list()   
    ins_max_col = list()
    ins_min_col = list()
    ins_std_col = list()
    for samp_time in time_col:
        tmp1 = [ '{0}_0_{1}_mean'.format(i,samp_time) for i in tmp_col]
        tmp2 = [ '{0}_0_{1}_sum'.format(i,samp_time) for i in tmp_col]
        tmp3 = [ '{0}_0_{1}_max'.format(i,samp_time) for i in tmp_col]
        tmp4 = [ '{0}_0_{1}_min'.format(i,samp_time) for i in tmp_col]
        tmp5 = [ '{0}_0_{1}_std'.format(i,samp_time) for i in tmp_col]
        ins_mean_col.append(tmp1.copy())
        ins_sum_col.append(tmp2.copy())
        ins_max_col.append(tmp3.copy())
        ins_min_col.append(tmp4.copy())
        ins_std_col.append(tmp5.copy())

    for i,dst_time in enumerate(time_col):
        df[ins_mean_col[i]] = df.rolling(dst_time,min_periods=1).agg(dict(zip(ins_col,['mean']*len(ins_col))))
        df[ins_sum_col[i]] = df.rolling(dst_time,min_periods=1).agg(dict(zip(ins_col,['sum']*len(ins_col))))
        df[ins_max_col[i]] = df.rolling(dst_time,min_periods=1).agg(dict(zip(ins_col,['max']*len(ins_col))))
        df[ins_min_col[i]] = df.rolling(dst_time,min_periods=1).agg(dict(zip(ins_col,['min']*len(ins_col))))
        df[ins_std_col[i]] = df.rolling(dst_time,min_periods=1).agg(dict(zip(ins_col,['std']*len(ins_col))))        

    df['y_7'] = df.rolling(FixedForwardWindowIndexer(window_size=window_size)).agg({label_later_col:'max'})
    df.dropna(inplace=True)
    df['label'] = (df['y_7']/(df[label_font_col])>1.02).astype(int)
    df['label'].value_counts()
    df.drop(future_drop,axis=1,inplace=True)
    df.dropna(inplace=True)

    dst_file_name = file_path.split('/')[-1]
    end_path = os.path.join(paras_path, dst_file_name)
    df.to_excel(end_path,index=False)